# The following notebook shows the Flickr api GIS project created by Nathan Wies and Kazuto Nishimori
## The aim of this code is to find interesting trends in spatial Flickr data

In [159]:
"""
Code written by Nathan Wies and Kazuto Nishimori
1/11/20
"""

#Initialize api

import USERAUTH
import flickr_api
from flickr_api.api import flickr
import geopandas as gpd
import pandas as pd
import json
import numpy as np

### Using the search method to pull data
#### Link to method documentation: https://www.flickr.com/services/api/flickr.photos.search.html

#### There is a pretty extensive list of spatial methods by which one can look up data which generally includes most parameters you could think of. All of these are posted at the link above.


##### bbox - 
A comma-delimited list of 4 values defining the Bounding Box of the area that will be searched.

The 4 values represent the bottom-left corner of the box and the top-right corner, minimum_longitude, minimum_latitude, maximum_longitude, maximum_latitude.

In [165]:
paris_pictures_json = flickr.photos.search(privacy_filter = 1, accuracy = 16, lat=48.857765, lon = 2.295672, radius = .005, format= "json", nojsoncallback = 1)
parispics = json.loads(paris_pictures_json)
print("The total number of pictures found within 5 km of the center of Paris is",
    parispics["photos"]["total"])
print(parispics)

The total number of pictures found within 5 km of the center of Paris is 56
{'photos': {'page': 1, 'pages': 1, 'perpage': 250, 'total': '56', 'photo': [{'id': '37328053084', 'owner': '17993950@N00', 'secret': '2484e22b1f', 'server': '4460', 'farm': 5, 'title': 'IMG_7035', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '33100036401', 'owner': '24827377@N00', 'secret': '15fb045ceb', 'server': '730', 'farm': 1, 'title': 'Dancing and singing Gospel songs', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '22758831916', 'owner': '33531758@N03', 'secret': 'f9c1e274b3', 'server': '5626', 'farm': 6, 'title': 'Panorámica de Paris, 2', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '22354010651', 'owner': '30150883@N04', 'secret': '9862635c55', 'server': '566', 'farm': 1, 'title': '', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '21205122724', 'owner': '35729004@N02', 'secret': 'e8a8c1e43a', 'server': '710', 'farm': 1, 'title': 'IMG_3230', 'ispublic': 1, 'isfriend': 0, 'i

In [155]:
paris_pictures_json = flickr.photos.search(privacy_filter = 1, accuracy = 16, lat=48.857765, lon = 2.295672, radius = .005, format= "json", nojsoncallback = 1)
parispics = json.loads(paris_pictures_json)


In [174]:
def findpicturefreq(lonmin, lonmax, latmin, latmax, size):
    df = pd.DataFrame(columns =["Latitude", "Longitude", "Total Pictures"])
    X = np.linspace(latmin, latmax, num=3)
    Y =  np.linspace(lonmin, lonmax, num=3)
    xv, yv = np.meshgrid(X, Y, sparse=False)
    print(np.shape(xv))
    for i in range(size):
        for j in range(size):
            print(xv[i][j])
            print(yv[i][j])
    return xv, yv

findpicturefreq(2.282772, 2.322956,48.861748, 48.886598, 3)

(3, 3)
48.861748
2.282772
48.874173
2.282772
48.886598
2.282772
48.861748
2.302864
48.874173
2.302864
48.886598
2.302864
48.861748
2.322956
48.874173
2.322956
48.886598
2.322956


In [156]:
paris_pictures_json

b'{"photos":{"page":1,"pages":1,"perpage":250,"total":"11","photo":[{"id":"42729146421","owner":"139132001@N03","secret":"b8746cbc11","server":"1723","farm":2,"title":"Portrait of Maria Eleonora of Brandenburg (Konigsberg, 1599-Stockholm, 1655)","ispublic":1,"isfriend":0,"isfamily":0},{"id":"17073504","owner":"46769392@N00","secret":"e0c6ae13f1","server":"11","farm":1,"title":"tour eiffel","ispublic":1,"isfriend":0,"isfamily":0},{"id":"26340025028","owner":"136515480@N04","secret":"3e692feedb","server":"4620","farm":5,"title":"La D\\u00e9fense sous la neige","ispublic":1,"isfriend":0,"isfamily":0},{"id":"39314707365","owner":"136515480@N04","secret":"b896887ee4","server":"4715","farm":5,"title":"La D\\u00e9fense sous la neige","ispublic":1,"isfriend":0,"isfamily":0},{"id":"39501071934","owner":"136515480@N04","secret":"018c834768","server":"4702","farm":5,"title":"La D\\u00e9fense sous la neige","ispublic":1,"isfriend":0,"isfamily":0},{"id":"26340021028","owner":"136515480@N04","secret

## Creating Dataframe from pulled data (parsing as json file)

In [135]:
"""
This method pulls data from the previous list. Unfortunatly this list is really long
so we NEED to check how many pings are acceptable or if there is a way to pull all of 
this data at once instead of sequentially, and if there is a limit to how much data 
we can pull per day.
"""
def create_df(pics):

    df = pd.DataFrame(columns =['id','latitude', 'longitude', 'accuracy', 'context', 'locality', 'county', 'region', 'country', 
                              'neighbourhood','title', "owner"])
    i=0
    for pic in pics["photos"]["photo"]:
        newfile = json.loads(flickr.photos.geo.getLocation(photo_id = pic["id"], format= "json", nojsoncallback = 1))
        df.loc[len(df)] = parse_json_data(newfile, pic["title"], pic["owner"])
        
        i+=1
        if(i>4):
            return df


def parse_json_data(output, title, owner):
    values = []
    output = output['photo']
    values.append(output['id'])
    output = output['location']
    values.append(output['latitude'])
    values.append(output['longitude'])
    values.append(output['accuracy'])
    values.append(output['context'])
    try:
        values.append(output['locality']['_content'])
    except:
        values.append("NaN")
    values.append(output['county']['_content'])
    values.append(output['region']['_content'])
    values.append(output['country']['_content'])
    values.append(output['neighbourhood']['_content'])
    values.append(title)
    values.append(owner)
    return values

df = create_df(parispics)
df      


,id,latitude,longitude,accuracy,context,locality,county,region,country,neighbourhood,title,owner
0,49374493921,48.871666,2.378611,16,0,Paris,Paris,Île-de-France,France,Belleville,2020-01-12_07-57-57,66944824@N05
1,49374162721,48.886822,2.343059,16,0,Paris,Paris,Île-de-France,France,Paris 18 Buttes-Montmartre,Combo enflammé,27420180@N08
2,49374322607,48.853252,2.347022,16,0,NaN,Paris,Île-de-France,France,,River Siene,7980350@N08
3,49373661968,48.853266,2.347027,16,0,NaN,Paris,Île-de-France,France,,Notre Dame Cathedral,7980350@N08
4,49374318272,48.852536,2.347975,16,0,NaN,Paris,Île-de-France,France,,Notre Dame After the Fire,7980350@N08
5,49374189552,48.893664,2.266595,16,0,Arrondissement de Nanterre,Hauts-de-Seine,Île-de-France,France,Neuilly-sur-Seine,La Seine au Pont de Courbevoie,18205690@N02
6,49373989221,48.859052,2.338070,16,0,Paris,Paris,Île-de-France,France,Paris 04 Ancien - Quartier Louvre,"Réflexion, le pont des arts à l'aube en novemb...",138486769@N02
7,49373505968,48.853502,2.346958,16,0,NaN,Paris,Île-de-France,France,,Notre Dame Cathedral,7980350@N08
8,49374102187,48.855394,2.345066,16,0,Paris,Paris,Île-de-France,France,Paris 11 Ancien - Quartier Palais-de-Justice,Stained Glass Cathedral,7980350@N08
9,49374099122,48.855394,2.345066,16,0,Paris,Paris,Île-de-France,France,Paris 11 Ancien - Quartier Palais-de-Justice,Above the Altar,7980350@N08


# Extra Stuff

In [ ]:
#this method finds all photos associated with a user and attempts to find the geodata
def toymethod():
    user = flickr_api.Person.findByUserName("tomquirkphoto")
    photos = user.getPhotos()
    for photo in photos:
        print(flickr.photos.geo.getLocation(photo_id = photo["id"]))

In [15]:
# importing the requests library 
import requests 
import pandas as pd
  
# api-endpoint 
file_url = "https://opendata.paris.fr/api/v2/catalog/datasets/logement-encadrement-des-loyers/exports/csv"
    
r = requests.get(file_url, stream = True) 
  
with open("datafileimport.csv","wb") as csv: 
    for chunk in r.iter_content(chunk_size=1024): 
  
         # writing one chunk at a time to pdf file 
         if chunk: 
             csv.write(chunk) 
                

df =  pd.read_csv('datafileimport.csv', sep=';'  , engine='python')
df

,id_zone,id_quartier,nom_quartier,piece,epoque,meuble_txt,ref,max,min,annee,ville,code_grand_quartier,geo_shape,geo_point_2d
0,5,2,Halles,2,Avant 1946,meublÃ©,27.3,32.8,19.1,2017,PARIS,7510102,"{""type"": ""Polygon"", ""coordinates"": [[[2.349365...","48.8622891081, 2.34489885831"
1,2,5,Gaillon,2,1946-1970,non meublÃ©,26.6,31.9,18.6,2017,PARIS,7510205,"{""type"": ""Polygon"", ""coordinates"": [[[2.336320...","48.8693066381, 2.33343180766"
2,2,5,Gaillon,3,Apres 1990,meublÃ©,27.8,33.4,19.5,2017,PARIS,7510205,"{""type"": ""Polygon"", ""coordinates"": [[[2.336320...","48.8693066381, 2.33343180766"
3,2,5,Gaillon,4,1971-1990,meublÃ©,24.9,29.9,17.4,2017,PARIS,7510205,"{""type"": ""Polygon"", ""coordinates"": [[[2.336320...","48.8693066381, 2.33343180766"
4,4,7,Mail,1,1946-1970,meublÃ©,33.0,39.6,23.1,2017,PARIS,7510207,"{""type"": ""Polygon"", ""coordinates"": [[[2.346838...","48.8680083374, 2.34469912743"
5,4,7,Mail,2,Avant 1946,non meublÃ©,27.3,32.8,19.1,2017,PARIS,7510207,"{""type"": ""Polygon"", ""coordinates"": [[[2.346838...","48.8680083374, 2.34469912743"
6,4,7,Mail,4,1971-1990,meublÃ©,25.9,31.1,18.1,2017,PARIS,7510207,"{""type"": ""Polygon"", ""coordinates"": [[[2.346838...","48.8680083374, 2.34469912743"
7,4,8,Bonne-Nouvelle,2,Apres 1990,non meublÃ©,25.4,30.5,17.8,2017,PARIS,7510208,"{""type"": ""Polygon"", ""coordinates"": [[[2.351518...","48.8671501183, 2.35008019041"
8,4,9,Arts-et-Metiers,3,Avant 1946,meublÃ©,27.8,33.4,19.5,2017,PARIS,7510309,"{""type"": ""Polygon"", ""coordinates"": [[[2.360209...","48.8664702895, 2.35708313106"
9,4,10,Enfants-Rouges,3,1971-1990,meublÃ©,25.8,31.0,18.1,2017,PARIS,7510310,"{""type"": ""Polygon"", ""coordinates"": [[[2.367101...","48.863887392, 2.36312330099"
